In [1]:
import pandas as pd

In [51]:
data = pd.read_csv('clicks_dataset_msk_20230101_20230725_spec.csv')
data.columns = ['request_id', 'object_id', 'target']

In [60]:
data

,request_id,object_id,target
0,1590973,718560,0
1,1234953,325828,1
2,1234953,135968,0
3,3326557,334526,1
4,3326557,541953,0
...,...,...,...
12979876,493850,545216,0
12979877,3008578,1128285,1
12979878,3008578,738151,0
12979879,4481384,413671,1


## Гипотеза - 1

### В рамках одинаковых связок (request_id - object_id) есть записи с разными таргетами 

In [97]:
stat_df = pd.DataFrame(data.groupby(['request_id', 'object_id'])['target'].value_counts())
stat_df.columns = ['cnt']
stat_df = stat_df.reset_index(level = ['request_id', 'object_id', 'target'])
stat_df

,request_id,object_id,target,cnt
0,0,458311,0,1
1,0,514656,1,1
2,1,474969,1,1
3,1,679728,0,1
4,2,126512,0,1
...,...,...,...,...
12977291,4520361,461228,0,1
12977292,4520362,477273,0,1
12977293,4520362,561093,1,1
12977294,4520363,120434,0,1


In [116]:
stat_t1 = stat_df[stat_df['target'] == 1][['request_id', 'object_id', 'cnt']]
stat_t1.columns = ['request_id', 'object_id', 'target_1_cnt']

stat_t0 = stat_df[stat_df['target'] == 0][['request_id', 'object_id', 'cnt']]
stat_t0.columns = ['request_id', 'object_id', 'target_0_cnt']

stat_full = stat_t1.merge(stat_t0, how = 'left', on = ['request_id', 'object_id'])

stat_full = stat_full.fillna(0)
stat_full[['target_1_cnt', 'target_0_cnt']] = stat_full[['target_1_cnt', 'target_0_cnt']].astype(int)

stat_out = stat_full[(stat_full['target_1_cnt'] > 0) &
                     (stat_full['target_0_cnt'] > 0)].reset_index(drop = True)

In [118]:
stat_out

,request_id,object_id,target_1_cnt,target_0_cnt
0,203168,564481,1,1
1,375422,559107,1,1
2,413553,508120,1,1
3,485280,103381,1,1
4,487181,798637,1,1
...,...,...,...,...
1404,4372226,802680,1,1
1405,4381632,80539,1,1
1406,4386727,88454,1,1
1407,4386728,522591,1,1


#### Всего в 1409 уникальной связке (request_id - object_id) есть хотя бы 1 как положительный, так и отрицательный таргет. Это могло как-то повлиять на некорректное поведение модели, но полученное кол-во связок слишком маленькое относительно всей выборки.

## Гипотеза - 2

### В рамках 1 object_id есть записи только с одним таргетом

In [135]:
stat_df = pd.DataFrame(data.groupby(['object_id'])['target'].value_counts())
stat_df.columns = ['cnt']
stat_df = stat_df.reset_index(level = ['object_id', 'target'])
stat_df

,object_id,target,cnt
0,0,1,10
1,1,1,1
2,2,1,2
3,3,1,1
4,4,1,2
...,...,...,...
1465324,1154535,1,1
1465325,1154536,1,1
1465326,1154537,1,1
1465327,1154538,1,1


In [136]:
stat_t1 = stat_df[stat_df['target'] == 1][['object_id', 'cnt']]
stat_t1.columns = ['object_id', 'target_1_cnt']

stat_t0 = stat_df[stat_df['target'] == 0][['object_id', 'cnt']]
stat_t0.columns = ['object_id', 'target_0_cnt']

stat_full = stat_t1.merge(stat_t0, how = 'left', on = ['object_id'])

stat_full = stat_full.fillna(0)
stat_full[['target_1_cnt', 'target_0_cnt']] = stat_full[['target_1_cnt', 'target_0_cnt']].astype(int)
stat_full['total'] = stat_full['target_1_cnt'] + stat_full['target_0_cnt']
stat_full['target_1_distr'] = round(stat_full['target_1_cnt'] / stat_full['total'], 3)

In [144]:
stat_full[stat_full['target_1_distr'] == 1].sort_values(by = 'total', ascending = False)

,object_id,target_1_cnt,target_0_cnt,total,target_1_distr
123917,135367,2838,0,2838,1.0
145998,161420,2698,0,2698,1.0
87464,91490,1947,0,1947,1.0
341096,356746,1926,0,1926,1.0
318857,334279,1750,0,1750,1.0
...,...,...,...,...,...
492050,532134,1,0,1,1.0
492048,532132,1,0,1,1.0
492043,532127,1,0,1,1.0
492042,532126,1,0,1,1.0


In [145]:
stat_full[(stat_full['target_1_distr'] == 1) &
          (stat_full['total'] > 1)].sort_values(by = 'total', ascending = False)

,object_id,target_1_cnt,target_0_cnt,total,target_1_distr
123917,135367,2838,0,2838,1.0
145998,161420,2698,0,2698,1.0
87464,91490,1947,0,1947,1.0
341096,356746,1926,0,1926,1.0
318857,334279,1750,0,1750,1.0
...,...,...,...,...,...
294875,310297,2,0,2,1.0
294876,310298,2,0,2,1.0
294877,310299,2,0,2,1.0
294884,310306,2,0,2,1.0


In [142]:
prop = round(stat_full[stat_full['target_1_distr'] == 1].shape[0] / stat_full.shape[0], 2)

print(f'[+] Доля уникальных объектов, у которых только таргет 1: {prop}')

[+] Доля уникальных объектов, у которых только таргет 1: 0.69


#### Если рассматривать выборку в рамках уникальных объектов (object_id), то у 69% записи только с положительным классом, при этом это не всегда 1 запись для объекта (~45% имеют больше 1 записи). При обучении на таких данных модель переобучится для этих объектов всегда предсказывать положительных класс вне зависимости от запроса. В результате чего на трейне будут хорошие метрики, а на бою данная модель не будет работать корректо.